In [5]:
import sys
import string
from datetime import datetime

ALPHABET_LOWER = string.ascii_lowercase
ALPHABET_UPPER = string.ascii_uppercase
ALPH_LEN = 26

def shift_char(c, shift):
    if c in ALPHABET_LOWER:
        idx = ALPHABET_LOWER.index(c)
        return ALPHABET_LOWER[(idx + shift) % ALPH_LEN]
    if c in ALPHABET_UPPER:
        idx = ALPHABET_UPPER.index(c)
        return ALPHABET_UPPER[(idx + shift) % ALPH_LEN]
    return c  # non-letter unchanged

def caesar(text, shift):
    return ''.join(shift_char(c, shift) for c in text)

def save_to_file(mode, plaintext, ciphertext, shift, default_name=None):
    """
    Save result to .txt with the requested formatted layout.
    mode: "ENCRYPT", "DECRYPT", or "AUTO_DECRYPT"
    plaintext: original plain text (or decrypted result for auto)
    ciphertext: original cipher text (or encrypted result)
    shift: integer shift used (or the discovered k for auto)
    """
    # Timestamp without seconds to match example format
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
    header = "==== Caesar Cipher Result ===="
    footer = "=" * 29

    result_text = (
        f"{header}\n"
        f"Date: {timestamp}\n"
        f"Mode: {mode}\n"
        f"Plaintext: {plaintext}\n"
        f"Shift: {shift}\n"
        f"Ciphertext: {ciphertext}\n"
        f"Notes: Shift used = {shift} (classic Caesar)\n"
        f"{footer}\n"
    )

    if default_name is None:
        default_name = f"caesar_{mode.lower()}_{datetime.now().strftime('%Y%m%d_%H%M')}.txt"

    name = input(f"Simpan hasil ke file (tekan Enter untuk '{default_name}'): ").strip()
    if not name:
        name = default_name

    try:
        with open(name, 'w', encoding='utf-8') as f:
            f.write(result_text)
        print(f"Hasil tersimpan ke '{name}'.\n")
        print("Berikut isi file:")
        print(result_text)
    except Exception as e:
        print("Gagal menyimpan file:", e)

def encrypt_flow():
    plain = input("Masukkan teks yang ingin dienkripsi:\n")
    while True:
        try:
            k = int(input("Masukkan shift (0-25): "))
            k = k % ALPH_LEN
            break
        except ValueError:
            print("Masukkan angka antara 0 sampai 25.")
    cipher = caesar(plain, k)
    print("\nHasil enkripsi:")
    print(cipher)

    save_to_file(mode="ENCRYPT", plaintext=plain, ciphertext=cipher, shift=k)
    return

def decrypt_flow():
    cipher = input("Masukkan teks yang ingin didekripsi:\n")
    while True:
        try:
            k = int(input("Masukkan shift (0-25): "))
            k = k % ALPH_LEN
            break
        except ValueError:
            print("Masukkan angka antara 0 sampai 25.")
    plain = caesar(cipher, -k)
    print("\nHasil dekripsi:")
    print("Ciphertext:", cipher)
    print("Plaintext :", plain)

    save_to_file(mode="DECRYPT", plaintext=cipher, ciphertext=plain, shift=k)
    return

def auto_decrypt_flow():
    cipher = input("Masukkan teks terenkripsi (akan dicoba semua shift 0..25):\n")
    candidates = []
    print("\nMencoba semua kemungkinan (0..25):\n")
    for k in range(ALPH_LEN):
        attempt = caesar(cipher, -k)
        candidates.append((k, attempt))
        print(f"[k={k:2}] {attempt}")
    print("\nJika salah satu terlihat benar, masukkan nomor k yang ingin disimpan.")
    while True:
        choice = input("Masukkan k pilihan (atau 'n' untuk batal): ").strip().lower()
        if choice == 'n':
            print("Batal menyimpan.")
            return
        try:
            k_choice = int(choice)
            if 0 <= k_choice < ALPH_LEN:
                selected = dict(candidates)[k_choice]
                print("\nTeks terpilih:")
                print(selected)

                save_to_file(mode=f"AUTO_DECRYPT", plaintext=selected, ciphertext=cipher, shift=k_choice)
                return
        except ValueError:
            pass
        print("Input tidak valid. Masukkan angka antara 0 dan 25 atau 'n'.")

def main():
    print("=== Caesar Cipher Tool ===")
    print("Pilih mode:")
    print("  1. Enkripsi")
    print("  2. Dekripsi (dengan kunci)")
    print("  3. Auto-decrypt (brute force semua kunci)")
    print("  0. Keluar")
    while True:
        choice = input("Masukkan pilihan (0-3): ").strip()
        if choice == '1':
            encrypt_flow()
            break
        elif choice == '2':
            decrypt_flow()
            break
        elif choice == '3':
            auto_decrypt_flow()
            break
        elif choice == '0':
            print("Keluar.")
            return
        else:
            print("Pilihan tidak valid. Coba lagi.")

if __name__ == "__main__":
    main()

=== Caesar Cipher Tool ===
Pilih mode:
  1. Enkripsi
  2. Dekripsi (dengan kunci)
  3. Auto-decrypt (brute force semua kunci)
  0. Keluar
Masukkan pilihan (0-3): 1
Masukkan teks yang ingin dienkripsi:
Kambing
Masukkan shift (0-25): 5

Hasil enkripsi:
Pfrgnsl
Simpan hasil ke file (tekan Enter untuk 'caesar_encrypt_20251031_1801.txt'): 
Hasil tersimpan ke 'caesar_encrypt_20251031_1801.txt'.

Berikut isi file:
==== Caesar Cipher Result ====
Date: 2025-10-31 18:01
Mode: ENCRYPT
Plaintext: Kambing
Shift: 5
Ciphertext: Pfrgnsl
Notes: Shift used = 5 (classic Caesar)



1. Caesar Cipher

Konsep: Pergeseran huruf tetap (shift)

Kelemahan:

Sangat mudah dipecahkan karena hanya ada 25 kemungkinan shift

Rentan brute force dan frekuensi analisis

In [4]:
from datetime import datetime

def vigenere_encrypt(plaintext, key):
    ciphertext = ""
    key = key.upper()
    plaintext = plaintext.upper()
    key_index = 0
    for char in plaintext:
        if char.isalpha():
            shift = ord(key[key_index]) - ord('A')
            ciphertext += chr((ord(char) - ord('A') + shift) % 26 + ord('A'))
            key_index = (key_index + 1) % len(key)
        else:
            ciphertext += char
    return ciphertext

def vigenere_decrypt(ciphertext, key):
    plaintext = ""
    key = key.upper()
    key_index = 0
    for char in ciphertext:
        if char.isalpha():
            shift = ord(key[key_index]) - ord('A')
            plaintext += chr((ord(char) - ord('A') - shift) % 26 + ord('A'))
            key_index = (key_index + 1) % len(key)
        else:
            plaintext += char
    return plaintext

# ===== MAIN PROGRAM =====
print("=== Vigenere Cipher ===")
mode = input("Mode (ENCRYPT/DECRYPT): ").upper()
text = input("Masukkan teks: ")
key = input("Masukkan key: ")

if mode == "ENCRYPT":
    result = vigenere_encrypt(text, key)
else:
    result = vigenere_decrypt(text, key)

print("\n==== Cipher Result ====")
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Mode:", mode)
print("Plaintext:", text if mode=="ENCRYPT" else result)
print("Key:", key)
print("Ciphertext:", result if mode=="ENCRYPT" else text)
print("Notes: Rentan terhadap analisis frekuensi jika kunci pendek")
print("======================")

=== Vigenere Cipher ===
Mode (ENCRYPT/DECRYPT): Decrypt
Masukkan teks: ANANF
Masukkan key: N

==== Cipher Result ====
Date: 2025-10-31 17:47:33
Mode: DECRYPT
Plaintext: NANAS
Key: N
Ciphertext: ANANF
Notes: Rentan terhadap analisis frekuensi jika kunci pendek


2. Vigenère Cipher

Konsep: Pergeseran huruf berbeda-beda sesuai key (kata kunci)

Kelemahan:

Jika key pendek, cipher masih rentan terhadap analisis frekuensi berulang (Kasiski examination)

Membutuhkan plaintext dan key yang presisi → lebih rumit untuk manual

Cipher bisa dipecahkan jika attacker mengetahui pola **key**

In [6]:
from datetime import datetime

def affine_encrypt(plaintext, a, b):
    ciphertext = ""
    plaintext = plaintext.upper()
    for char in plaintext:
        if char.isalpha():
            ciphertext += chr(((a * (ord(char) - ord('A')) + b) % 26) + ord('A'))
        else:
            ciphertext += char
    return ciphertext

def affine_decrypt(ciphertext, a, b):
    plaintext = ""
    a_inv = pow(a, -1, 26)  # inverse modulo 26
    for char in ciphertext:
        if char.isalpha():
            plaintext += chr(((a_inv * ((ord(char) - ord('A')) - b)) % 26) + ord('A'))
        else:
            plaintext += char
    return plaintext

# ===== MAIN PROGRAM =====
print("=== Affine Cipher ===")
mode = input("Mode (ENCRYPT/DECRYPT): ").upper()
text = input("Masukkan teks: ")
a = int(input("Masukkan a (coprime dengan 26): "))
b = int(input("Masukkan b: "))

if mode == "ENCRYPT":
    result = affine_encrypt(text, a, b)
else:
    result = affine_decrypt(text, a, b)

print("\n==== Cipher Result ====")
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Mode:", mode)
print("Plaintext/Ciphertext:", text if mode=="ENCRYPT" else result)
print("Key: a={}, b={}".format(a, b))
print("Ciphertext/Plaintext:", result if mode=="ENCRYPT" else text)
print("Notes: Memerlukan a coprime dengan 26; rentan analisis frekuensi")
print("======================")

=== Affine Cipher ===
Mode (ENCRYPT/DECRYPT): encrypt
Masukkan teks: Pulpen
Masukkan a (coprime dengan 26): 7
Masukkan b: 9

==== Cipher Result ====
Date: 2025-10-31 18:03:10
Mode: ENCRYPT
Plaintext/Ciphertext: Pulpen
Key: a=7, b=9
Ciphertext/Plaintext: KTIKLW
Notes: Memerlukan a coprime dengan 26; rentan analisis frekuensi


3. Affine Cipher

Konsep: Enkripsi matematis
E(x) = (a∗x+b) mod 26

Kelemahan:

Masih termasuk monoalphabetic substitution, sehingga dapat dianalisis dengan frekuensi huruf

Harus memilih a coprime 26 → jika tidak, dekripsi tidak mungkin

Tidak aman untuk ciphertext panjang karena polanya tetap

In [9]:
from datetime import datetime

def create_playfair_matrix(key):
    key = "".join(dict.fromkeys(key.upper().replace("J", "I")))  # hapus duplikat dan gabung J ke I
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    matrix = []
    for char in key:
        if char not in matrix:
            matrix.append(char)
    for char in alphabet:
        if char not in matrix:
            matrix.append(char)
    # buat matriks 5x5
    return [matrix[i*5:(i+1)*5] for i in range(5)]

def find_position(matrix, char):
    for i, row in enumerate(matrix):
        if char in row:
            return i, row.index(char)
    return None

def process_plaintext(text):
    text = text.upper().replace("J", "I").replace(" ", "")
    result = ""
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) else "X"
        if a == b:
            result += a + "X"
            i += 1
        else:
            result += a + b
            i += 2
    return result

def playfair_encrypt(text, matrix):
    ciphertext = ""
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)
        if r1 == r2:
            ciphertext += matrix[r1][(c1+1)%5] + matrix[r2][(c2+1)%5]
        elif c1 == c2:
            ciphertext += matrix[(r1+1)%5][c1] + matrix[(r2+1)%5][c2]
        else:
            ciphertext += matrix[r1][c2] + matrix[r2][c1]
    return ciphertext

def playfair_decrypt(text, matrix):
    plaintext = ""
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)
        if r1 == r2:
            plaintext += matrix[r1][(c1-1)%5] + matrix[r2][(c2-1)%5]
        elif c1 == c2:
            plaintext += matrix[(r1-1)%5][c1] + matrix[(r2-1)%5][c2]
        else:
            plaintext += matrix[r1][c2] + matrix[r2][c1]
    return plaintext

# ===== MAIN PROGRAM =====
print("=== Playfair Cipher ===")
mode = input("Mode (ENCRYPT/DECRYPT): ").upper()
text = input("Masukkan teks: ")
key = input("Masukkan key: ")

matrix = create_playfair_matrix(key)
processed_text = process_plaintext(text)

if mode == "ENCRYPT":
    result = playfair_encrypt(processed_text, matrix)
else:
    result = playfair_decrypt(processed_text, matrix)

print("\n==== Cipher Result ====")
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Mode:", mode)
print("Plaintext:", text if mode=="ENCRYPT" else result)
print("Key:", key)
print("Ciphertext:", result if mode=="ENCRYPT" else text)
print("Notes: Huruf J digabung dengan I; digraph diproses dengan matriks 5x5")
print("======================")

=== Playfair Cipher ===
Mode (ENCRYPT/DECRYPT): Encrypt
Masukkan teks: Padang
Masukkan key: Pisang

==== Cipher Result ====
Date: 2025-10-31 18:23:03
Mode: ENCRYPT
Plaintext: Padang
Key: Pisang
Ciphertext: INLDPE
Notes: Huruf J digabung dengan I; digraph diproses dengan matriks 5x5


4. Playfair Cipher

Konsep: Digraph substitution menggunakan matriks 5×5

Kelemahan:

Lebih aman dari Caesar/Vigenère, tapi masih bisa dianalisis dengan analisis digraf jika ciphertext panjang

Enkripsi/dekripsi manual lebih rumit

Key harus sama persis untuk dekripsi

In [11]:
from datetime import datetime
import numpy as np

def mod26_inverse_matrix(matrix):
    det = int(round(np.linalg.det(matrix)))  # determinan matriks
    det_inv = pow(det % 26, -1, 26)  # invers modulo 26
    adj = np.round(det * np.linalg.inv(matrix)).astype(int)  # adjoin matrix
    return (det_inv * adj) % 26

def process_text(text):
    text = text.upper().replace(" ", "")
    if len(text) % 2 != 0:  # jika ganjil, tambahkan X
        text += "X"
    nums = [ord(c)-ord('A') for c in text]
    return nums

def encrypt_hill(text, key_matrix):
    nums = process_text(text)
    ciphertext = ""
    for i in range(0, len(nums), 2):
        block = np.array(nums[i:i+2])
        cipher_block = key_matrix.dot(block) % 26
        ciphertext += ''.join(chr(n + ord('A')) for n in cipher_block)
    return ciphertext

def decrypt_hill(text, key_matrix):
    nums = process_text(text)
    key_inv = mod26_inverse_matrix(key_matrix)
    plaintext = ""
    for i in range(0, len(nums), 2):
        block = np.array(nums[i:i+2])
        plain_block = key_inv.dot(block) % 26
        plaintext += ''.join(chr(int(n) + ord('A')) for n in plain_block)
    return plaintext

# ===== MAIN PROGRAM =====
print("=== Hill Cipher 2x2 ===")
mode = input("Mode (ENCRYPT/DECRYPT): ").upper()
text = input("Masukkan teks: ")

print("Masukkan matriks key 2x2:")
k11 = int(input("Elemen [1,1]: "))
k12 = int(input("Elemen [1,2]: "))
k21 = int(input("Elemen [2,1]: "))
k22 = int(input("Elemen [2,2]: "))

key_matrix = np.array([[k11, k12], [k21, k22]])

if mode == "ENCRYPT":
    result = encrypt_hill(text, key_matrix)
else:
    result = decrypt_hill(text, key_matrix)

print("\n==== Cipher Result ====")
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Mode:", mode)
print("Plaintext:", text if mode=="ENCRYPT" else result)
print("Key:")
print(key_matrix)
print("Ciphertext:", result if mode=="ENCRYPT" else text)
print("Notes: Matriks harus 2x2, determinan coprime dengan 26; tambahkan X jika panjang teks ganjil")
print("======================")

=== Hill Cipher 2x2 ===
Mode (ENCRYPT/DECRYPT): encrypt
Masukkan teks: Lintang
Masukkan matriks key 2x2:
Elemen [1,1]: 3
Elemen [1,2]: 2
Elemen [2,1]: 3
Elemen [2,2]: 5

==== Cipher Result ====
Date: 2025-10-31 18:27:20
Mode: ENCRYPT
Plaintext: Lintang
Key:
[[3 2]
 [3 5]]
Ciphertext: XVZEANMD
Notes: Matriks harus 2x2, determinan coprime dengan 26; tambahkan X jika panjang teks ganjil


5. Hill Cipher

Konsep: Blok huruf dienkripsi dengan perkalian matriks modulo 26

Kelemahan:

Jika key diketahui, mudah dipecahkan dengan sistem persamaan linear

Harus memastikan determinan matriks coprime 26 → kalau salah, dekripsi gagal

Dekripsi manual rumit jika matriks besar